# https://blog.akquinet.de/2017/10/25/predicting-house-prices-on-kaggle-a-gentle-introduction-to-data-science-part-ii/

Some information like basement may be empty indicating there is no basement for the building

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_log_error, mean_absolute_error
from sklearn.neural_network import MLPClassifier
from sklearn.impute import SimpleImputer

In [13]:
house_data = pd.read_csv("train.csv").drop("Id", axis=1)
test_data = pd.read_csv("test.csv")
house_data.head()


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [14]:
from sklearn.decomposition import PCA

In [15]:
house_data.shape

(1460, 80)

In [16]:
test_data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


In [17]:
cols_with_missing_train = [col for col in house_data.columns
                     if house_data[col].isnull().any()]
print(cols_with_missing_train)

['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']


In [18]:
cols_with_missing_test = [col for col in test_data.columns
                     if test_data[col].isnull().any()]
print(cols_with_missing_test)

['MSZoning', 'LotFrontage', 'Alley', 'Utilities', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType']


In [21]:
string_data = house_data.select_dtypes(include=['object'])
le = preprocessing.LabelEncoder()
ordinal_encoder = preprocessing.OrdinalEncoder(handle_unknown='use_encoded_value',
                                 unknown_value=-1)
labels=pd.DataFrame(columns = house_data.columns)

ordinal_encoder.fit(house_data[string_data])   
house_data[string_data] = ordinal_encoder.transform(house_data[string_data])
test_data[string_data] = ordinal_encoder.transform(test_data[string_data])

ValueError: Boolean array expected for the condition, not object

In [8]:
imputer = SimpleImputer(strategy="mean")
house_data[cols_with_missing_train] = imputer.fit_transform(house_data[cols_with_missing_train])

ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: 'Grvl'

In [ ]:
test_data[cols_with_missing_test] = imputer.fit_transform(test_data[cols_with_missing_test])

In [ ]:
cols_with_missing_test = [col for col in house_data.columns
                     if house_data[col].isnull().any()]
print(cols_with_missing_test)

In [ ]:
house_data[cols_with_missing_test].isnull().sum() / house_data.shape[0] * 100

Project Plans:
1. Use PCA to identify key features
2. Use ordinal encoding for categories and see the performance
3. Replace missing values with more accurate values instead of deleting them
4. Numerical features that are secretly categorical

In [ ]:
import bisect
le_classes = le.classes_.tolist()
bisect.insort_left(le_classes, '<unknown>')
le.classes_ = le_classes

for column in string_data:
    test_data[column] = ordinal_encoder.transform(test_data[column])

In [ ]:
house_data['SalePrice']

In [ ]:
clf = MLPClassifier(solver='adam', early_stopping = True, hidden_layer_sizes=(46, 20), random_state=0, max_iter = 2000, batch_size=200, shuffle=True)
clf.fit(house_data.iloc[:, :46].values ,house_data['SalePrice'].values)

In [ ]:
pred = clf.predict(test_data[house_data.iloc[:, :46].columns].values)
pred

In [ ]:
output = pd.DataFrame(columns=['Id', 'SalePrice'])
output['Id'] = test_data['Id']
output['SalePrice'] = pred

np.unique(output['SalePrice'], return_index=False, return_inverse=False, return_counts=True, axis=None)

In [ ]:
output.to_csv("out_2.csv", index=None)

In [ ]:
pca = PCA()
pca.fit_transform(house_data)
pca.feature_names_in_

In [ ]:
initial_feature_names = house_data.columns

In [ ]:
most_important = [np.abs(pca.components_[i]).argmax() for i in range(36)]
most_important_names = [initial_feature_names[most_important[i]] for i in range(36)]

In [ ]:
dic = {'PC{}'.format(i): most_important_names[i] for i in range(36)}

df = pd.DataFrame(dic.items())
df

In [ ]:
train_2 = house_data[df[1]]
train_2_final = train_2.drop('SalePrice', axis = 1)

In [ ]:
clf = MLPClassifier(solver='adam', early_stopping = True, hidden_layer_sizes=(40, 10), random_state=1, max_iter = 2000, batch_size=500, shuffle=True)
clf.fit(train_2_final.values ,train_2['SalePrice'].values)

In [ ]:
test_data[train_2_final.columns]

In [ ]:
pred = clf.predict(test_data[train_2_final.columns].values)
pred

In [ ]:
output = pd.DataFrame(columns=['Id', 'SalePrice'])
output['Id'] = test_data['Id']
output['SalePrice'] = pred

np.unique(output['SalePrice'], return_index=False, return_inverse=False, return_counts=True, axis=None)

In [ ]:
output.to_csv("out_8.csv", index=None)